# Deploy NGC pytorch model to SageMaker

For a description of the model you can look here: https://ngc.nvidia.com/catalog/model-scripts/nvidia:ssd_for_pytorch

Let's get started by downloading an SSD model from NVIDIA

In [1]:
!wget https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt

--2020-05-08 20:44:19--  https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 54.186.237.130, 52.38.124.212
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|54.186.237.130|:443... connected.
HTTP request sent, awaiting response... 302 
Location: https://s3.us-west-2.amazonaws.com/prod-model-registry-ngc-bucket/org/nvidia/models/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt?response-content-disposition=attachment%3B%20filename%3D%22nvidia_ssdpyt_fp32_190826.pt%22&response-content-type=application%2Foctet-stream&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEKT%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLXdlc3QtMiJGMEQCIDTOcIMEjEOZCz8fE8A9VS52BuQxLfxHV5HjD2XMImILAiBBGPC5D%2FBlfMhB%2Bpe4bYj7mYlpOnoeNb4%2FIHSuAQ7Gpiq9Awjd%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAIaDDc4OTM2MzEzNTAyNyIMsVQ6WZTk70s2icTCKpED0yPq8kHJUawmVePhsCIr4MUulWsp7dMgEPEvCZp1Ca3BirYXCGkMUo4E82%2Bxf5uMsGdEK2fvxB2rlw5vERMNeDg4abxn

In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
import tarfile
import torch
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from utils import dboxes300_coco, Encoder


sess = sagemaker.Session()
role = get_execution_role()
bucket = sess.default_bucket() # can replace with your own s3 bucket! 

Now that we have imported the necessary packages, we need to take the model file we downloaded and make it into a tarball. This is the format that Sagemaker needs the model to be in to deploy it to an endpoint.

In [3]:
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('nvidia_ssdpyt_fp32_190826.pt')

Our next step is to upload this data to s3 

In [4]:
modelpath = sess.upload_data(
    path='model.tar.gz', bucket=bucket,
    key_prefix='sagemaker-pytorch/input')

Now we are going to use the built in Jupyter magic command to write our transform_script.py. This file is going to tell our endpoint how to load our model, handle input data, and make predictions. We first initialize the model architecture, which is a SSD300 object detector based on a ResNet50 backbone. 

In [5]:
%%writefile transform_script.py

import torch
import os
import numpy as np

import torch.nn as nn
from torchvision.models.resnet import resnet18, resnet34, resnet50, resnet101, resnet152

class ResNet(nn.Module):
    def __init__(self, backbone='resnet50', backbone_path=None):
        super().__init__()
        if backbone == 'resnet18':
            backbone = resnet18(pretrained=not backbone_path)
            self.out_channels = [256, 512, 512, 256, 256, 128]
        elif backbone == 'resnet34':
            backbone = resnet34(pretrained=not backbone_path)
            self.out_channels = [256, 512, 512, 256, 256, 256]
        elif backbone == 'resnet50':
            backbone = resnet50(pretrained=not backbone_path)
            self.out_channels = [1024, 512, 512, 256, 256, 256]
        elif backbone == 'resnet101':
            backbone = resnet101(pretrained=not backbone_path)
            self.out_channels = [1024, 512, 512, 256, 256, 256]
        else:  # backbone == 'resnet152':
            backbone = resnet152(pretrained=not backbone_path)
            self.out_channels = [1024, 512, 512, 256, 256, 256]
        if backbone_path:
            backbone.load_state_dict(torch.load(backbone_path))


        self.feature_extractor = nn.Sequential(*list(backbone.children())[:7])

        conv4_block1 = self.feature_extractor[-1][0]

        conv4_block1.conv1.stride = (1, 1)
        conv4_block1.conv2.stride = (1, 1)
        conv4_block1.downsample[0].stride = (1, 1)

    def forward(self, x):
        x = self.feature_extractor(x)
        return x

class SSD300(nn.Module):
    def __init__(self, backbone=ResNet('resnet50')):
        super().__init__()

        self.feature_extractor = backbone

        self.label_num = 81  # number of COCO classes
        self._build_additional_features(self.feature_extractor.out_channels)
        self.num_defaults = [4, 6, 6, 6, 4, 4]
        self.loc = []
        self.conf = []

        for nd, oc in zip(self.num_defaults, self.feature_extractor.out_channels):
            self.loc.append(nn.Conv2d(oc, nd * 4, kernel_size=3, padding=1))
            self.conf.append(nn.Conv2d(oc, nd * self.label_num, kernel_size=3, padding=1))

        self.loc = nn.ModuleList(self.loc)
        self.conf = nn.ModuleList(self.conf)
        self._init_weights()

    def _build_additional_features(self, input_size):
        self.additional_blocks = []
        for i, (input_size, output_size, channels) in enumerate(zip(input_size[:-1], input_size[1:], [256, 256, 128, 128, 128])):
            if i < 3:
                layer = nn.Sequential(
                    nn.Conv2d(input_size, channels, kernel_size=1, bias=False),
                    nn.BatchNorm2d(channels),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(channels, output_size, kernel_size=3, padding=1, stride=2, bias=False),
                    nn.BatchNorm2d(output_size),
                    nn.ReLU(inplace=True),
                )
            else:
                layer = nn.Sequential(
                    nn.Conv2d(input_size, channels, kernel_size=1, bias=False),
                    nn.BatchNorm2d(channels),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(channels, output_size, kernel_size=3, bias=False),
                    nn.BatchNorm2d(output_size),
                    nn.ReLU(inplace=True),
                )

            self.additional_blocks.append(layer)

        self.additional_blocks = nn.ModuleList(self.additional_blocks)

    def _init_weights(self):
        layers = [*self.additional_blocks, *self.loc, *self.conf]
        for layer in layers:
            for param in layer.parameters():
                if param.dim() > 1: nn.init.xavier_uniform_(param)

    # Shape the classifier to the view of bboxes
    def bbox_view(self, src, loc, conf):
        ret = []
        for s, l, c in zip(src, loc, conf):
            ret.append((l(s).view(s.size(0), 4, -1), c(s).view(s.size(0), self.label_num, -1)))

        locs, confs = list(zip(*ret))
        locs, confs = torch.cat(locs, 2).contiguous(), torch.cat(confs, 2).contiguous()
        return locs, confs

    def forward(self, x):
        x = self.feature_extractor(x)

        detection_feed = [x]
        for l in self.additional_blocks:
            x = l(x)
            detection_feed.append(x)

        # Feature Map 38x38x4, 19x19x6, 10x10x6, 5x5x6, 3x3x4, 1x1x4
        locs, confs = self.bbox_view(detection_feed, self.loc, self.conf)

        # For SSD 300, shall return nbatch x 8732 x {nlabels, nlocs} results
        return locs, confs

# this function tells the endpoitn how to make predictions and how to package them to send back
def predict_fn(input_data, model):
    # run prediction
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    with torch.no_grad():
        pred = model(input_data)
    pred_dict = {'pred1':pred[0].detach().cpu().numpy(),'pred2':pred[1].detach().cpu().numpy()}
    return pred_dict
        
# this function loads our model from s3, or if that fails, from the NGC repo
def model_fn(model_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SSD300(backbone=ResNet('resnet50'))
    try:
        model_weights = torch.load(os.path.join(model_dir, 'nvidia_ssdpyt_fp32_190826.pt'), map_location='cpu')['model']
        model.to('cpu')
        model.load_state_dict(model_weights)
    except:
        print('using fallback model loading')
        os.system('wget https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt')
        model_weights = torch.load(os.path.join('nvidia_ssdpyt_fp32_190826.pt'), map_location='cpu')['model']
        model.to('cpu')
        model.load_state_dict(model_weights)
    model.eval()
    return model 

# this function handles our input data and reshapes it back into an image
def input_fn(request_body, request_content_type):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if(request_content_type == 'application/x-npy'):
        try:
            input_data = np.frombuffer(request_body, dtype=int)
        except:
            input_data = np.array(request_body, dtype=int)
    try:
        input_data = torch.tensor(np.reshape(input_data,(1,3,300,300)), dtype=torch.float32, device=device) # this needs to be a torch tensor 
    except:
        input_data = torch.tensor(np.reshape(input_data[16:],(1,3,300,300)), dtype=torch.float32, device=device) # this needs to be a torch tensor 
    return input_data

Overwriting transform_script.py


Use the SageMaker PyTorchModel to instantiate and deploy the model onto a GPU instance

In [7]:
pytorch_model = PyTorchModel(model_data=modelpath, role=role,
                             entry_point='transform_script.py',
                             framework_version='1.4.0')

predictor = pytorch_model.deploy(instance_type='ml.g4dn.4xlarge',
                                 initial_instance_count=1,
                                 wait=True,
                                 endpoint_name='torch-ssd-ngc-model')

---------------!

### Download, convert and send images for inference

In [8]:
uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg',
    'http://images.cocodataset.org/val2017/000000037777.jpg',
    'http://images.cocodataset.org/val2017/000000252219.jpg'
]

img = imread('https://upload.wikimedia.org/wikipedia/commons/2/25/Postmen_Office_Room.jpg')
img  = resize(img, (300,300,3))
img = np.array(img, dtype=np.int64)
img.shape

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(300, 300, 3)

In [9]:
result = predictor.predict(img.tobytes())
result

array({'pred1': array([[[ 0.38952032, -0.02861576, -0.04215011, ..., -0.14958134,
          0.00432905, -0.12932765],
        [ 0.09118789,  0.04212354, -0.14801054, ...,  0.19097322,
          0.26461983,  0.18199176],
        [-2.2925906 , -1.8833055 , -1.5824505 , ..., -0.08838636,
         -0.13544835,  2.1892743 ],
        [-0.87224966, -0.71586657, -0.696971  , ..., -0.16525826,
          1.9327695 , -0.3432234 ]]], dtype=float32), 'pred2': array([[[ 8.178115  ,  8.464893  ,  8.17106   , ...,  7.6592464 ,
          7.431782  ,  7.3305855 ],
        [ 1.4540417 ,  1.6984062 ,  1.5541288 , ...,  2.1266382 ,
          2.3925123 ,  2.0987399 ],
        [-0.7406039 , -0.6521972 , -0.5834959 , ..., -0.9978748 ,
         -0.95666707, -1.0767772 ],
        ...,
        [-0.48175845, -0.5065458 , -0.3991172 , ..., -0.12805915,
         -0.19128259, -0.16708916],
        [-0.5146159 , -0.625941  , -0.57805604, ..., -1.3090948 ,
         -1.3886985 , -1.2481947 ],
        [-0.13825768, -0.0

In [ ]:
# dboxes = dboxes300_coco()
# encoder = Encoder(dboxes)
# ploc, plabel = [val.float() for val in prediction]
# encoded = encoder.decode_batch(ploc, plabel, criteria=0.5, max_output=20)

## Cleanup

Remove model and delete endpoint

In [10]:
!rm nvidia_ssdpyt_fp32_190826.pt

In [ ]:
predictor.delete_endpoint()